In [ ]:
import pandas as pd

df = pd.read_csv("../data/evaluation/all_dependencies.csv")

df_sample = df.sample(n=100)

df_sample.to_csv("../data/evaluation/test_dependencies.csv", index=False)

In [ ]:
import json

data_file = "../data/evaluation/config5/all_dependencies_all.json"
output_file = "../data/evaluation/config5/all_dependencies_all_updated.json"

with open(data_file, "r", encoding="utf-8") as src:
    data = json.load(src)


for entry in data:
    context = entry["context"][:3]
    context_str = "\n\n".join([x["content"] for x in context])
    
    entry["context"] = context
    entry["context_str"] = context_str

with open(output_file, "a", encoding="utf-8") as dest:
    json.dump(data, dest, indent=2)


**Get index of dependencies for which no context could be retrieved**

In [ ]:
import json

data_file = "../data/evaluation/config5/all_dependencies_all.json"

with open(data_file, "r", encoding="utf-8") as src:
    data = json.load(src)

    print(len(data))

    print(data[0]["context"][0].keys())

    for x in data:
        context = x["context"]
        if len(context) == 0:
            print(x["index"])

**Get index of dependencies for which the response format cannot be load as json**

In [8]:
import json

data_file = "../data/evaluation/without/all_dependencies_without_llama3:8b.json"

target_indices = []

with open(data_file, "r", encoding="utf-8") as src:
    data = json.load(src)

    print(len(data))

    print(data[0]["context"][0].keys())

    for entry in data:
        response = entry["response"]
        
        try:
            response_dict = json.loads(response)
        except json.JSONDecodeError:
            target_indices.append(entry["index"])
            print("JSONDecodeError Index: ", entry["index"])
            try:
                isDependency = response_dict["isDependency"]
            except KeyError as error:
                print("IsDependency missing: ", entry["index"])

        except KeyError:
            target_indices.append(entry["index"])
            print("KeyError Index: ", entry["index"])

        
target_indices

500
dict_keys(['content', 'score', 'index', 'id'])
JSONDecodeError Index:  31
JSONDecodeError Index:  39
JSONDecodeError Index:  58
JSONDecodeError Index:  94
JSONDecodeError Index:  108
JSONDecodeError Index:  210
JSONDecodeError Index:  301
JSONDecodeError Index:  321
JSONDecodeError Index:  327
JSONDecodeError Index:  338
JSONDecodeError Index:  422
JSONDecodeError Index:  432
JSONDecodeError Index:  447


[31, 39, 58, 94, 108, 210, 301, 321, 327, 338, 422, 432, 447]

**Run generation pipeline for specific indices**

In [1]:
from prompt_settings import PrompSettingsFactory
from typing import Dict, List
from generator import GeneratorFactory, GeneratorEngine
from dotenv import load_dotenv
from util import load_config
from tqdm import tqdm
import backoff
import json


@backoff.on_exception(backoff.expo, Exception, max_tries=10)
def generate(generator: GeneratorEngine, messages: List) -> str:
    response = generator.generate(messages=messages)

    if not response:
        raise Exception("Response is empty.")
    
    try:
        json.loads(response)
    except json.JSONDecodeError:
        raise Exception("Response cannot be loaded as JSON.")

    return response


def run_generation(config: Dict, target_index: List[int]) -> None:
    
    print("Data file: ", config["data_file"])
    print("With RAG: ", config["with_rag"])
    

    with open(config["data_file"], "r", encoding="utf-8") as src:
        data = json.load(src)

    prompt_settings = PrompSettingsFactory.get_prompt_settings(tool_name=config["tool_name"])

    generator = GeneratorFactory().get_generator(
        model_name=config["model_name"], 
        temperature=config["temperature"]
    )

    for entry in tqdm(data, total=len(data), desc="Processing entries"):
        
        if not entry["index"] in target_index:
            continue

        print("Process index: ", entry["index"])

        if config["with_rag"]:
            query_str = prompt_settings.query_prompt.format(
                context_str=entry["context_str"], 
                task_str=entry["task_str"],
                format_str=prompt_settings.get_format_prompt()
            )
        else:
            query_str =f"{entry['task_str']}\n\n{prompt_settings.get_format_prompt()}"

        messages = [
            {
                "role": "system", 
                "content": entry["system_str"]
            },
            {
                "role": "user",
                "content": query_str
            }
        ]

        response = generate(
            generator=generator,
            messages=messages
        )

        #entry["responses"] = [response]
        entry["response"] = response
        
    output_file = config["data_file"]
    with open(output_file, "w", encoding="utf-8") as dest:
        json.dump(data, dest, indent=2)
            

config_file = "../generation_config.toml"
env_file = "../.env"

load_dotenv(dotenv_path=env_file)

# load config
config = load_config(config_file=config_file)
run_generation(config=config, target_index=[125, 133, 181, 192, 238, 264, 268, 282, 285, 486])

Data file:  ../data/results/config1/all_dependencies_all_gpt-3.5-turbo-0125.json
With RAG:  True


[09/02/24 19:06:18] INFO     Initialize CfgNet prompt settings.                              ]8;id=74639;file:///home/ssimon/projects/cval/src/prompt_settings.py\prompt_settings.py]8;;\:]8;id=646236;file:///home/ssimon/projects/cval/src/prompt_settings.py#192\192]8;;\

                    INFO     GPT (gpt-3.5-turbo-0125) generator initialized.                        ]8;id=93133;file:///home/ssimon/projects/cval/src/generator.py\generator.py]8;;\:]8;id=532207;file:///home/ssimon/projects/cval/src/generator.py#88\88]8;;\

Processing entries:   0%|          | 0/500 [00:00<?, ?it/s]

[09/02/24 19:06:21] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=167481;file:///home/ssimon/projects/cval/cval_env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=356178;file:///home/ssimon/projects/cval/cval_env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\
                             "HTTP/1.1 200 OK"                                                                     

...

Processing entries:  57%|█████▋    | 283/500 [00:23<00:26,  8.21it/s]

[09/02/24 19:06:43] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=775098;file:///home/ssimon/projects/cval/cval_env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=492345;file:///home/ssimon/projects/cval/cval_env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\
                             "HTTP/1.1 200 OK"                                                                     

Processing entries:  57%|█████▋    | 286/500 [00:25<00:36,  5.87it/s]

[09/02/24 19:06:46] INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions          ]8;id=605980;file:///home/ssimon/projects/cval/cval_env/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=369693;file:///home/ssimon/projects/cval/cval_env/lib/python3.10/site-packages/httpx/_client.py#1026\1026]8;;\
                             "HTTP/1.1 200 OK"                                                                     

Processing entries: 100%|██████████| 500/500 [00:28<00:00, 17.40it/s]


In [25]:
import json

model_names = ["gpt-4o-2024-05-13", "gpt-3.5-turbo-0125", "llama3:8b", "llama3:70b"]

updated_file = "../data/results/config4/all_dependencies_all_updated.json"
data_file = "../data/results/config4/all_dependencies_all_llama3:70b.json"


with open(updated_file, "r", encoding="utf-8") as src:
    updated_data = json.load(src)


with open(data_file, "r", encoding="utf-8") as src:
    data = json.load(src)



for entry in updated_data:
    index = entry["index"]
    print(index)
    data[index] = entry


with open(data_file, "w", encoding="utf-8") as dest:
    json.dump(data, dest, indent=2)

125
133
181
192
238
264
268
282
285
486
